# Workshop DL02: Convolutional Neural Networks

Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009.
https://www.cs.toronto.edu/~kriz/cifar.html

## Worked Example: CIFAR-10

In [2]:
import tensorflow as tf
import numpy as np
import pickle

/Users/evan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
DATA_PATH = "cifar-10-batches-py/"
N_PIXELS = 32
N_CHANNELS = 3
N_CLASSES = 10

In [4]:
def convert_images(raw_images):
    """
    Converts an array of flattened images (None ,3072) into
    (None, 32, 32, 3)
    """
    images = np.array(raw_images, dtype=float) / 255.0
    images = images.reshape([-1, N_CHANNELS, N_PIXELS, N_PIXELS])
    images = images.transpose([0, 2, 3, 1])
    return images

In [ ]:
def convert_labels(raw_labels):
    """
    Converts the unit array of labels to 10-one-hot.
    """
    labels 

In [5]:
def load_batch(path):
    """
    Loads a single CIFAR-10 batch from the given path.
    """
    with open(path, mode="rb") as f:
        raw = pickle.load(f, encoding="bytes")
        raw_images = raw[b'data']
        labels = raw[b'labels']
    return raw_images, labels

In [80]:
images, labels = load_batch(DATA_PATH + "data_batch_1")
images = convert_images(images)

In [81]:
# (1) input
with tf.name_scope("input"):
    X = tf.placeholder(tf.float32,
        shape=[None, N_PIXELS, N_PIXELS, N_CHANNELS])

In [85]:
# (2) network architecture
with tf.name_scope("conv-1"):
    conv1 = tf.layers.conv2d(X, 10, [11,11], padding="same",
        activation=tf.nn.relu)
    conv1_flat = tf.reshape(conv1, [-1, 10240])
    
with tf.name_scope("dense-1"):
    dense1 = tf.layers.dense(conv1_flat, 200, activation=tf.nn.relu)

In [86]:
# (3) output
with tf.name_scope("output"):
    y_pre_softmax = tf.layers.dense(dense1, N_CLASSES,
        activation=None)
    y_ = tf.nn.softmax(y_pre_softmax)

In [101]:
# (4) loss
with tf.name_scope("loss"):
    y = tf.placeholder(tf.int64, shape=[None, ])
    y_one_hot = tf.one_hot(indices=y, depth=N_CLASSES)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=y_pre_softmax, labels=y_one_hot)

In [102]:
# (5) optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [109]:
# (6) train
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    _, l = sess.run([optimizer, loss], feed_dict={X: [images[0]], y: [labels[0]]})
    print(l)

[2.849996]


In [104]:
# (7) test
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output = sess.run([y_], feed_dict={X: [images[0]]})
    print(output)

[array([[0.09956872, 0.06013068, 0.08568408, 0.08924558, 0.09540431,
        0.13081455, 0.11452979, 0.12063329, 0.09729027, 0.10669877]],
      dtype=float32)]


In [99]:
labels

[6,
 9,
 9,
 4,
 1,
 1,
 2,
 7,
 8,
 3,
 4,
 7,
 7,
 2,
 9,
 9,
 9,
 3,
 2,
 6,
 4,
 3,
 6,
 6,
 2,
 6,
 3,
 5,
 4,
 0,
 0,
 9,
 1,
 3,
 4,
 0,
 3,
 7,
 3,
 3,
 5,
 2,
 2,
 7,
 1,
 1,
 1,
 2,
 2,
 0,
 9,
 5,
 7,
 9,
 2,
 2,
 5,
 2,
 4,
 3,
 1,
 1,
 8,
 2,
 1,
 1,
 4,
 9,
 7,
 8,
 5,
 9,
 6,
 7,
 3,
 1,
 9,
 0,
 3,
 1,
 3,
 5,
 4,
 5,
 7,
 7,
 4,
 7,
 9,
 4,
 2,
 3,
 8,
 0,
 1,
 6,
 1,
 1,
 4,
 1,
 8,
 3,
 9,
 6,
 6,
 1,
 8,
 5,
 2,
 9,
 9,
 8,
 1,
 7,
 7,
 0,
 0,
 6,
 9,
 1,
 2,
 2,
 9,
 2,
 6,
 6,
 1,
 9,
 5,
 0,
 4,
 7,
 6,
 7,
 1,
 8,
 1,
 1,
 2,
 8,
 1,
 3,
 3,
 6,
 2,
 4,
 9,
 9,
 5,
 4,
 3,
 6,
 7,
 4,
 6,
 8,
 5,
 5,
 4,
 3,
 1,
 8,
 4,
 7,
 6,
 0,
 9,
 5,
 1,
 3,
 8,
 2,
 7,
 5,
 3,
 4,
 1,
 5,
 7,
 0,
 4,
 7,
 5,
 5,
 1,
 0,
 9,
 6,
 9,
 0,
 8,
 7,
 8,
 8,
 2,
 5,
 2,
 3,
 5,
 0,
 6,
 1,
 9,
 3,
 6,
 9,
 1,
 3,
 9,
 6,
 6,
 7,
 1,
 0,
 9,
 5,
 8,
 5,
 2,
 9,
 0,
 8,
 8,
 0,
 6,
 9,
 1,
 1,
 6,
 3,
 7,
 6,
 6,
 0,
 6,
 6,
 1,
 7,
 1,
 5,
 8,
 3,
 6,
 6,
 8,
 6,
 8,
 4,
 6,
 6,
